In [2]:
!pip install -q gdown
!gdown --id 1vAuTUfkeRX045AMhUPcguxwov2dqAVtJ --output my_dataset.zip

import gzip
import shutil

with gzip.open('my_dataset.zip', 'rb') as f_in:
    with open('my_dataset', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

import tarfile

with tarfile.open("my_dataset", "r:") as tar:
    tar.extractall("my_dataset_extracted")

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1vAuTUfkeRX045AMhUPcguxwov2dqAVtJ

but Gdown can't. Please check connections and permissions.


FileNotFoundError: [Errno 2] No such file or directory: 'my_dataset.zip'

In [ ]:
# Cell 1: Install required packages
!pip install gigagan-pytorch==0.2.20 fvcore torch torchvision validators

In [ ]:
import os
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from gigagan_pytorch import GigaGAN
import logging
import json
import time
import torchvision.utils as vutils
from fvcore.nn import parameter_count
import torch.nn.utils.prune as prune
from torch.cuda.amp import autocast

In [ ]:
UNCONDITIONAL = True
IMAGE_SIZE = 256
BATCH_SIZE = 4
TRAINING_STEPS = 1000
FINETUNE_STEPS = 100
DATA_PATH = "my_dataset_extracted/GigaGAN_cond_imagenet256"
SAVE_DIR = "gigagan_pruned_results"
MODEL_FOLDER = "gigagan_pruning_checkpoints"
RESULTS_FOLDER = "gigagan_pruning_results"

os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs("profiling_results", exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    handlers=[logging.FileHandler("gigagan_training.log"), logging.StreamHandler()]
)
logger = logging.getLogger("gigagan")

In [ ]:
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

def load_imagenet_dataset(path):
    dataset = datasets.ImageFolder(root=path, transform=transform)
    logger.info(f"Loaded dataset with {len(dataset)} samples from {path}")
    return dataset

def collate_images_only(batch):
    images, _ = zip(*batch)
    return torch.stack(images)

In [ ]:
def profile_dataloader(dataloader, num_batches=10):
    logger.info(f"Profiling dataloader for {num_batches} batches...")
    batch_times = []
    use_cuda = torch.cuda.is_available()
    if use_cuda:
        torch.cuda.reset_peak_memory_stats()
        start_memory = torch.cuda.memory_allocated() / (1024 * 1024)
    else:
        start_memory = 0

    start = time.time()
    for i, images in enumerate(dataloader):
        if i == num_batches:
            break
    total_time = time.time() - start
    avg_time = total_time / num_batches
    peak_memory = torch.cuda.max_memory_allocated() / (1024 * 1024) if use_cuda else 0

    stats = {
        "avg_batch_time_sec": avg_time,
        "peak_memory_mb": peak_memory,
        "start_memory_mb": start_memory
    }

    with open("profiling_results/dataloader_stats.json", "w") as f:
        json.dump(stats, f, indent=2)

    logger.info(f"Dataloader avg time: {avg_time:.4f}s, peak memory: {peak_memory:.2f} MB")
    return stats

In [ ]:
def setup_model():
    logger.info("Setting up GigaGAN (Unconditional)...")
    gan = GigaGAN(
        train_upsampler=True,
        generator=dict(
            dim=32,
            style_network=dict(dim=64, depth=4),
            image_size=IMAGE_SIZE,
            input_image_size=64,
            unconditional=UNCONDITIONAL
        ),
        discriminator=dict(
            dim_capacity=16,
            dim_max=512,
            image_size=IMAGE_SIZE,
            multiscale_input_resolutions=(128,),
            num_skip_layers_excite=4,
            unconditional=UNCONDITIONAL
        ),
        learning_rate=1e-5,
        amp=True,
        model_folder=MODEL_FOLDER,
        results_folder=RESULTS_FOLDER
    ).cuda()

    total_params = sum(p.numel() for p in gan.parameters())
    trainable_params = sum(p.numel() for p in gan.parameters() if p.requires_grad)
    model_size_mb = sum(p.numel() * p.element_size() for p in gan.parameters()) / 1024**2

    logger.info(f"Model initialized: {total_params:,} params ({model_size_mb:.2f} MB)")
    return gan

In [ ]:
def train_model(gan, dataloader, steps, save_name="model-final.ckpt"):
    gan.set_dataloader(dataloader)
    logger.info(f"Training for {steps} steps (AMP enabled)")
    with autocast():
        gan(steps=steps, grad_accum_every=8)
    save_path = os.path.join(MODEL_FOLDER, save_name)
    gan.save(save_path)
    logger.info(f"Model saved to {save_path}")


In [ ]:
def count_params(model, label):
    gen_params = sum(p.numel() for p in model.unwrapped_G.parameters())
    disc_params = sum(p.numel() for p in model.unwrapped_D.parameters())
    total = gen_params + disc_params
    logger.info(f"{label} Generator: {gen_params/1e6:.2f}M, Discriminator: {disc_params/1e6:.2f}M, Total: {total/1e6:.2f}M")

def prune_gigagan_model(model, amount=0.3, target="both"):
    logger.info(f"Pruning GigaGAN ({target}) with {amount*100:.0f}% sparsity")
    if target in ["generator", "both"]:
        for _, module in model.unwrapped_G.named_modules():
            if isinstance(module, torch.nn.Conv2d):
                prune.ln_structured(module, name="weight", amount=amount, n=1, dim=0)
    if target in ["discriminator", "both"]:
        for _, module in model.unwrapped_D.named_modules():
            if isinstance(module, torch.nn.Conv2d):
                prune.ln_structured(module, name="weight", amount=amount, n=1, dim=0)
    logger.info("Pruning complete.")

def remove_pruning_reparam(model, target="both"):
    logger.info("Stripping pruning reparametrizations...")
    for net in ([model.unwrapped_G] if target == "generator" else [model.unwrapped_D, model.unwrapped_G]):
        for _, module in net.named_modules():
            if isinstance(module, torch.nn.Conv2d) and hasattr(module, "weight_mask"):
                prune.remove(module, "weight")

In [ ]:
def generate_and_save_images(gan, num_images=3):
    input_size = gan.unwrapped_G.input_image_size
    for i in range(num_images):
        noise = torch.randn(1, 3, input_size, input_size).cuda()
        with torch.no_grad(), autocast():
            img = gan.generate(lowres_image=noise)[0].cpu()
        img = torch.nan_to_num(img, nan=0.0).clamp(-1, 1)
        img = (img * 0.5 + 0.5)
        vutils.save_image(img, f"{SAVE_DIR}/generated_{i}.png")
        logger.info(f"Saved: {SAVE_DIR}/generated_{i}.png")

In [ ]:
def count_nonzero_params(model, label):
    def count_nonzero(p):
        return (p != 0).sum().item()

    gen_nonzero = sum(count_nonzero(p) for p in model.unwrapped_G.parameters() if p.requires_grad)
    disc_nonzero = sum(count_nonzero(p) for p in model.unwrapped_D.parameters() if p.requires_grad)
    total = gen_nonzero + disc_nonzero

    logger.info(f"{label} Generator: {gen_nonzero/1e6:.2f}M nonzero, Discriminator: {disc_nonzero/1e6:.2f}M nonzero, Total: {total/1e6:.2f}M nonzero")
    return gen_nonzero, disc_nonzero

In [ ]:
def train_gigagan():
    dataset = load_imagenet_dataset(DATA_PATH)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, collate_fn=collate_images_only)
    profile_dataloader(dataloader)
    gan = setup_model()
    count_params(gan, "Initial (Unpruned)")
    train_model(gan, dataloader, steps=TRAINING_STEPS, save_name="model-unpruned.ckpt")

# ---------------------------- Phase 2: Prune + Fine-tune ----------------------------
def prune_and_finetune_gigagan():
    dataset = load_imagenet_dataset(DATA_PATH)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, collate_fn=collate_images_only)
    gan = setup_model()
    gan.load(os.path.join(MODEL_FOLDER, "model-unpruned.ckpt"))
    count_params(gan, "Before Pruning")
    prune_gigagan_model(gan, amount=0.3, target="both")
    remove_pruning_reparam(gan, target="both")
    count_params(gan, "After Pruning + Reparam Removal")
    gan.unwrapped_G.load_state_dict(gan.unwrapped_G.state_dict())  # safe sync for EMA
    train_model(gan, dataloader, steps=FINETUNE_STEPS, save_name="model-pruned.ckpt")
    generate_and_save_images(gan)

def iterative_pruning_workflow(iterations=5, initial_prune=0.1):
    dataset = load_imagenet_dataset(DATA_PATH)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, collate_fn=collate_images_only)
    profile_dataloader(dataloader)

    gan = setup_model()
    train_model(gan, dataloader, steps=TRAINING_STEPS, save_name="model-unpruned.ckpt")
    gan.load(os.path.join(MODEL_FOLDER, "model-unpruned.ckpt"))

    cumulative_prune = 1.0
    for i in range(iterations):
        prune_step = 1 - initial_prune
        cumulative_prune *= prune_step
        logger.info(f"Iteration {i+1}: Pruning to cumulative {(1-cumulative_prune)*100:.1f}%")

        prune_gigagan_model(gan, amount=initial_prune, target="both")
        remove_pruning_reparam(gan, target="both")
        count_params(gan, f"Post-Prune Iteration {i+1}")

        gan.unwrapped_G.load_state_dict(gan.unwrapped_G.state_dict())
        train_model(gan, dataloader, steps=FINETUNE_STEPS, save_name=f"model-pruned-{i+1}.ckpt")

    generate_and_save_images(gan)

In [ ]:
gan = setup_model()
# gan.load(os.path.join(MODEL_FOLDER, "model-unpruned.ckpt"))
# count_nonzero_params(gan, "After Training")



Generator: 43.71M
Discriminator: 30.74M




In [ ]:
train_gigagan()

A100 GPU detected, using flash attention if input tensor is on cuda


Generator: 43.71M
Discriminator: 30.74M




<ipython-input-9-aea9fa4cc978>:4: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
  0%|          | 1/1000 [00:00<?, ?it/s]/usr/lib/python3.11/contextlib.py:105: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)


G: -3.39 | MSG: -14.02 | VG: 0.00 | D: 5.51 | MSD: 15.46 | VD: 0.00 | GP: 0.00 | SSL: 14.49 | CL: 0.00 | MAL: 0.00


  2%|▏         | 21/1000 [00:58<46:30,  2.85s/it]

G: -281.25 | MSG: -203.91 | VG: 0.00 | D: 276.11 | MSD: 204.89 | VD: 0.00 | GP: 27868.06 | SSL: 13.34 | CL: 0.00 | MAL: 0.00


...

G: -2733.28 | MSG: -158.94 | VG: 0.00 | D: 2734.12 | MSD: 160.54 | VD: 0.00 | GP: 28503.82 | SSL: 13.24 | CL: 0.00 | MAL: 0.00


 44%|████▍     | 441/1000 [20:05<26:50,  2.88s/it]

G: -2839.30 | MSG: -154.89 | VG: 0.00 | D: 2840.56 | MSD: 156.51 | VD: 0.00 | GP: 28207.96 | SSL: 12.96 | CL: 0.00 | MAL: 0.00


 45%|████▍     | 449/1000 [20:29<25:11,  2.74s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 512.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 382.88 MiB is free. Process 51716 has 39.17 GiB memory in use. Of the allocated memory 35.77 GiB is allocated by PyTorch, and 2.89 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
prune_and_finetune_gigagan()
gan = setup_model()
gan.load(os.path.join(MODEL_FOLDER, "model-unpruned.ckpt"))
count_nonzero_params(gan, "After Training")